In [ ]:
import os
import pandas as pd
import json
import notebook_utils.notebook_helpers as utils
from cuopt_sh_client import CuOptServiceSelfHostClient

# Priority Routing
## Capacitated Vehicle Routing Problem with prize collection.

Loyalty (or Preferred) customer programs help companies to reward repeat customers and enhance their overall business offering. While the best possible customer service is always the goal, loyalty programs provide a mechanism for reinforcing the relationship with the customers that drive business revenue.

In this scenario we have couple of deliveries out of which only few can be made at the cost of other, and depedning on loyality of a customer or may be cost of the delivery can be a priority. This problem will consider cost of the task/order.

### Problem Details:
- 8 Locations each with an associated demand
    - 1 Distribution Center 
        - distribution center demand: [0]
        - hours of operation: [0,24]
    - 7 Service Locations
        - demand for deliveries: [1, 1, 1, 1, 1, 1, 1]
        - delivery time windows: [[9,10],[9,10],[9,10],[10,11],[10,11],[10,11],[9,10]]
        - service location service times: [ 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
        - loyalty program member: [1, 0, 0, 0, 1, 0, 1]

- 3 Delivery vehicles each with an associated capacity
    - 3 delivery vehicles
        - capacity for deliveries: [3, 3, 3]
        

Below we visualize the delivery locations with respect to the distribution center. The cost from all locations to all other locations (a cost matrix) will be required for optimization. To see an example of cost matrix generation from map data or a waypoint graph, refer to the [cost_matrix_creation.ipynb](cost_matrix_creation.ipynb) notebook.  For the purpose of this simple example we will omit the cost matrix calculation.

In [ ]:
location_names       = [      "DC",          "A",    "B",    "C",    "D",    "E",    "F",    "G"  ]
location_coordinates = [     [4, 4],        [1, 3], [8, 1], [2, 1], [6, 7], [0, 2], [7, 6], [5, 3] ]
location_coordinates_df = pd.DataFrame(location_coordinates, columns=['xcord', 'ycord'], index=location_names)
utils.gen_plot(location_coordinates_df).show()

# Initialize cuOpt Service Client and cuOpt Problem Data

In [ ]:
ip = "0.0.0.0"
port = 5000

client = CuOptServiceSelfHostClient(
    ip=ip,
    port=port
)

cuopt_problem_data = {}

### Cost Matrix : Primary

The cost matrix dictates the cost of travel between locations of interest.  The cost itself can be anything relevant to the user.  In this case we are using a business defined cost objective as a primary cost matrix and a secondary time matrix to verify our time based constraints. 

Here is the cost(business metric) matrix corresponding to the locations above:

In [ ]:
business_metric_cost_matrix = [
    [0.0, 3.1, 5.0, 3.6, 3.6, 4.5, 3.6, 1.4],
    [3.1, 0.0, 7.3, 2.2, 6.4, 1.4, 6.7, 4.0],
    [5.0, 7.3, 0.0, 6.0, 6.3, 8.1, 5.1, 3.6],
    [3.6, 2.2, 6.0, 0.0, 7.2, 2.2, 7.1, 3.6],
    [3.6, 6.4, 6.3, 7.2, 0.0, 7.8, 1.4, 4.1],
    [4.5, 1.4, 8.1, 2.2, 7.8, 0.0, 8.1, 5.1],
    [3.6, 6.7, 5.1, 7.1, 1.4, 8.1, 0.0, 3.6],
    [1.4, 4.0, 3.6, 3.6, 4.1, 5.1, 3.6, 0.0]
]

### Cost Matrix : Secondary

Here is the constraint checking (time) secondary matrix:

In [ ]:
constraint_checking_time_matrix = [
    [0.00, 0.39, 0.63, 0.45, 0.45, 0.55, 0.45, 0.18 ],
    [0.39, 0.00, 0.90, 0.28, 0.80, 0.18, 0.84, 0.50 ],
    [0.63, 0.90, 0.00, 0.75, 0.79, 1.00, 0.64, 0.45 ],
    [0.45, 0.28, 0.75, 0.00, 0.90, 0.28, 0.88, 0.45 ],
    [0.45, 0.80, 0.79, 0.90, 0.00, 0.96, 0.18, 0.51 ],
    [0.55, 0.18, 1.00, 0.28, 0.96, 0.00, 1.00, 0.64 ],
    [0.45, 0.84, 0.64, 0.88, 0.18, 1.00, 0.00, 0.45 ],
    [0.18, 0.50, 0.45, 0.45, 0.51, 0.64, 0.45, 0.00 ]
]

### Deliveries

Setup the delivery data

In [ ]:
delivery_location_data = {
    "task_location":          [i+1 for i in range(len(location_names)-1)], # designate zeroth location as start and return points for fleet
    "task_ids":               ["a", "b", "c", "d", "e", "f", "g"],
    "delivery_demand":        [1,  1,  1,  1,  1,  1,  1 ],
    "location_earliest_time": [9,  9,  9,  10, 10, 10, 9 ],
    "location_latest_time":   [10, 10, 10, 11, 11, 11, 10],
    "required_service_time":  [1,  2,  1,  2,  1,  2,  1 ],
    "loyalty_member":         [0,  1,  0,  1,  0,  1,  0 ],
    "prizes":                 [1,  15,  1,  15,  1,  15,  1],
}
print(delivery_location_data)

### Set Cost Matrix

Dispatch cost matrix to server

In [ ]:
cuopt_problem_data["cost_matrix_data"] = {
        "cost_matrix": {
            "0": business_metric_cost_matrix
        }
    }

### Set Secondary Cost Matrix

Dispatch secondary cost matrix to server

In [ ]:
cuopt_problem_data["travel_time_matrix_data"] = {
        "data": {
            "0": constraint_checking_time_matrix
        }
    }

### Set Vehicle Data
Dispatch vehicle data to server

In [ ]:
n_vehicles = 3
vehicle_capacity = 3 # As per problem statement, all vehicles have capacities of 3

cuopt_problem_data["fleet_data"] = {
        "vehicle_locations": [[0,0]] * n_vehicles,
        "capacities": [[vehicle_capacity] * n_vehicles],
        "vehicle_time_windows": [[5, 20]] * n_vehicles
}

### Set Task Data
Dispatch task data to server

In [ ]:
cuopt_problem_data["task_data"] = {
        "task_locations": delivery_location_data["task_location"],
        "task_ids": delivery_location_data["task_ids"],
        "demand": [delivery_location_data["delivery_demand"]],
        "task_time_windows": list(zip(delivery_location_data["location_earliest_time"],
                                          delivery_location_data["location_latest_time"])),
        "service_times": delivery_location_data["required_service_time"],
}

### Set CuOpt Solver Configuration

In [ ]:
cuopt_problem_data["solver_config"] = {
        "time_limit": 5
    }

In [ ]:
print( json.dumps(cuopt_problem_data, indent=4))

### Attempt to obtain optimized routes
We can attempt to solve this problem as stated, but as previously discussed it is not feasible within the specified target time windows

In [ ]:
# Solve the problem

solver_response = client.get_optimized_routes(
    cuopt_problem_data
)

solver_resp = solver_response["response"]

if "solver_response" not in solver_resp.keys():
    print("cuOpt was unable to find a solution.")

cuOpt is unable to find a feasible solution.  As previously discussed we would like to allow the deliveries to exceed the latest time windows by using soft time windows

### Initial Solution

Lets enable prize collection, which will let solver deliver partial set of orders compared to al the orders.

#### Add prizes 

With this, we can prioritize order/customers by providing higher prizes to such jobs compared to others.

In [ ]:
cuopt_problem_data["task_data"]["prizes"] = delivery_location_data["prizes"]

#### Re-optimize 

In [ ]:
# Solve the problem
solver_response = client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

if solver_resp["status"] == 0: 
    solver_resp_df = utils.get_solution_df(solver_resp)
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    print(solver_resp_df)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

It can be observed that the tasks with higher prizes were chosen over other tasks

In [ ]:
solution_data_priority = utils.get_solution_df(solver_resp)
solution_data_priority['route'] = [location_names[i] for i in solution_data_priority['route'].to_list()]
solution_data_priority = solution_data_priority.set_index('route')
solution_data_priority


SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: MIT
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.